In [32]:
import pandas as pd
import numpy as np
import re
import os
import folium

## PD Set Options

In [33]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 800)

## Read in and setup data

In [34]:
### Read in data ###
df = pd.read_csv('broward_redfin_2023-02-16-07-22-00.csv')

In [35]:
### Rename columns ###
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [36]:
### Data check ###
print(f'Total sales: {len(df):,}')

Total sales: 2,782


## Data validation

In [37]:
### Remove NaNs from 'SOLD DATE' ###
df['SOLD DATE'] = df['SOLD DATE'].fillna('Not provided')
print(f'{len(df):,}')

2,782


In [38]:
### Filter to just the last month's data ###
x = df['SOLD DATE'].str.startswith('December')
df = df[x]
print(f'{len(df):,}')

873


In [39]:
### Ensure that only 'PROPERTY TYPE' Condo/Co-op is in the data ###
df = df.loc[df['PROPERTY TYPE'] == 'Condo/Co-op']
print(f'{len(df):,}')

873


In [40]:
### Sort properties by sale price, with highest sale price at the top ###
df = df.sort_values(by='PRICE',ascending=False)
### Create a price column formatted as currency ###
df['PRICE_AS_CURRENCY'] = df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df['BEDS'] = df['BEDS'].apply(lambda x: '{:,.0f}'.format(x))
df['BATHS'] = df['BATHS'].apply(lambda x: '{:,.0f}'.format(x))
df['YEAR BUILT'] = df['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df['PRICE_SQUARE_FEET_AS_CURRENCY'] = df['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [41]:
### Insert different colors for top 10 sales vs. the rest ###
df['COLOR'] = ''
### Create RANK column ###
df['RANK'] = 0
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 10, 'orange', 'blue')

# Data Checks

## Print formatting

In [42]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [43]:
### Convert 'YEAR BUILT' back to integer ###
df['YEAR BUILT'] = pd.to_numeric(df['YEAR BUILT'])

In [53]:
print(f"{ME}{BR}{df.loc[df['PRICE'].idxmax()]['LOCATION']}, {df.loc[df['PRICE'].idxmax()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df.loc[df['PRICE'].idxmin()]['LOCATION']}, {df.loc[df['PRICE'].idxmin()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmax()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmin()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df.loc[df['YEAR BUILT'].idxmax()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmax()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['YEAR BUILT'].idxmax()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df.loc[df['YEAR BUILT'].idxmin()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmin()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1804 | Price $4,600,000 | $2,052 psf | Year built: 2022
Least Expensive
Sunrise Lakes, 2600 W Sunrise Lakes Dr #308 | Price $50,000 | $78 psf | Year built: 1973
Highest Price Per Square Foot
CORONET TOWERS CONDO, 2444 Polk St #108 | Price $1,690,000 | $2,864 psf | Year built: 1967
Lowest Price Per Square Foot
Sunrise Lakes, 2600 W Sunrise Lakes Dr #308 | Price $50,000 | $78 psf | Year built: 1973
Newest
Four Seasons, 525 N Ft Lauderdale Bch Blvd #1804 | Price $4,600,000 | $2,052 psf | Year built: 2022
Oldest
Coral Cove, 2720 Yacht Club Blvd Unit 8B | Price $270,000 | $403 psf | Year built: 1955


## Data locater (if needed)

In [56]:
df.loc[df['PRICE'] == 4600000]['URL']

2450    https://www.redfin.com/FL/Fort-Lauderdale/525-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1804/home/144171260
Name: URL, dtype: object

## Data correction (if needed)

In [46]:
df.at[184,'PRICE'] = (189000)
df.at[184,'$/SQUARE FEET'] = (227)

df.at[2307,'PRICE'] = (270000)
df.at[2307,'$/SQUARE FEET'] = (347)

df.at[229,'PRICE'] = (195000)
df.at[229,'$/SQUARE FEET'] = (191)

## Summary Info

In [55]:
print('SALES INFO')
print(f'Number of sales: {len(df)}')
print('--------')
print(f'Total sale price: ${df["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df["YEAR BUILT"].mean()}')
print('------------------------------------------------')
print('BEDS & BATHS')
print(f'Most beds: {df["BEDS"].max()}')
print('----------')
print(f'Fewest beds: {df["BEDS"].min()}')
print('----------')
print(f'Most baths: {df["BATHS"].max()}')
print('----------')
print(f'Fewest baths: {df["BATHS"].min()}')

SALES INFO
Number of sales: 873
--------
Total sale price: $268,824,521
--------
Median sale price: $215,000
--------
Max sale price: $4,600,000
--------
Min sale price: $50,000
------------------------------------------------
PSF INFO
Max price per square foot: $2,864
--------
Min price per square foot: $78
--------
Median price per square foot: $214
------------------------------------------------
CONDO AGES
Newest building: 2022
----------
Oldest building: 1955
----------
Average building age: 1980.5841924398626
------------------------------------------------
BEDS & BATHS
Most beds: 4
----------
Fewest beds: 0
----------
Most baths: 4
----------
Fewest baths: 0


## Days on market calculator

In [57]:
from datetime import datetime, timedelta

date1 = datetime(2021, 6, 10) ## Earlier date (list date)
date2 = datetime(2022, 12, 22) ## Later date (sold date)

delta = date2 - date1
num_days = delta.days

print(num_days)

560


# Map Stuff

## HTML Popup Formatter

In [49]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [50]:
### Drop the columns ###
df = df.drop(columns=columns_drop)

In [51]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

## Make Map

In [52]:
### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Broward January Condo Sales")
m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m